# Tutorial - Text Mining - Classification 

We will predict the category of discussion posts in a newsgroup.

**The unit of analysis is a discussion post**

### Import common packages

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mssur\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\mssur\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
import pandas as pd
import numpy as np
import nltk
# nltk.download('averaged_perceptron_tagger') # you only need to run this once
from nltk.stem import WordNetLemmatizer 
from nltk import pos_tag, word_tokenize
from nltk.corpus import wordnet
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import CountVectorizer

np.random_seed = 1

### Load data

In [4]:
news = pd.read_csv('news.csv')
news.shape

(597, 5)

In [5]:
news.head(5)

,TEXT,graphics,hockey,medical,newsgroup
0,I have a few reprints left of chapters from my...,1,0,0,graphics
1,"gnuplot, etc. make it easy to plot real valued...",1,0,0,graphics
2,Article-I.D.: snoopy.1pqlhnINN8k1 References: ...,1,0,0,graphics
3,"Hello, I am looking to add voice input capabil...",1,0,0,graphics
4,I recently got a file describing a library of ...,1,0,0,graphics


# lemmatisation code

In [6]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(word):
    """Map POS tag to first character used by WordNetLemmatizer"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"A": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize_text(text):
    """Lemmatize input text"""
    lemmatized_text = []
    for word in nltk.word_tokenize(text):
        pos = get_wordnet_pos(word)
        lemmatized_text.append(lemmatizer.lemmatize(word, pos=pos))
    return " ".join(lemmatized_text)

# Load news dataset
news = news

# Lemmatize news headline column
news["TEXT"] = news["TEXT"].apply(lemmatize_text)

# Print the first 5 rows of the lemmatized dataset
print(news.head())

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\mssur\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mssur\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
news["TEXT"]

0      I have a few reprint left of chapter from my b...
1      gnuplot , etc . make it easy to plot real valu...
2      Article-I.D . : snoopy.1pqlhnINN8k1 References...
3      Hello , I am looking to add voice input capabi...
4      I recently got a file describing a library of ...
                             ...                        
592    carl @ SOL1.GPS.CALTECH.EDU ( Carl J Lydick ) ...
593    In article < 1qmlgaINNjab @ hp-col.col.hp.com ...
594    Article-I.D . : kestrel.1993Apr16.172052.27843...
595    In article < 1qmlgaINNjab @ hp-col.col.hp.com ...
596    I have a 42 yr old male friend , misdiagnosed ...
Name: TEXT, Length: 597, dtype: object

### Check for missing values

In [8]:
news[['TEXT']].isna().sum()

TEXT    0
dtype: int64

## Assign the input variable to X and the target variable to y

In [9]:
X = news['TEXT']

This is a multi-class classification problem. There are three categories we will predict:<br>
Whether a post is "graphics," "hockey," or "medical" related

In [10]:
y = news['newsgroup']
y.unique()

array(['graphics', 'hockey', 'medical'], dtype=object)

In [11]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(y)
print(le.classes_)
y = le.transform(y)
y

['graphics' 'hockey' 'medical']


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

## Split the data

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [13]:
X_train.shape, y_train.shape

((417,), (417,))

In [14]:
X_test.shape, y_test.shape

((180,), (180,))

In [15]:
X_train.head(5)

336    In article < 1993Apr14.235849.15397 @ amc.com ...
196    Article-I.D . : oasys.32850 Expires : 30 Apr 9...
216    In < 1993Apr5.185633.17843 @ ists.ists.ca > dc...
205    Article-I.D . : auvm.93095.210625MWEINTR Refer...
407    I am looking for publically accessible source ...
Name: TEXT, dtype: object

In [16]:
y_train[:5]

array([1, 0, 1, 1, 2])

## Sklearn: Text preparation

For simplicity (and focus), we will not do any text cleaning or preprocessing. We will just use the raw text as input to the model. See the text mining fundamentals tutorial for more details on text cleaning and preprocessing.

In [17]:
#TfidfVectorizer includes pre-processing, tokenization, filtering stop words
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(stop_words='english', lowercase=True, token_pattern="[^\W\d_]+")

X_train = tfidf_vect.fit_transform(X_train)

**Notice in the previous step that we use `fit_transform` on TRAIN. When we transform the TEST data, we need to use `transform` only. This enables us to keep the number of columns (features) the same across the data sets. Otherwise, they WILL be different, and no model will work!**

In [18]:
# Perform the TfidfVectorizer transformation
# Be careful: We are using the train fit to transform the test data set. Otherwise, the test data 
# features will be very different and match the train set!!!

X_test = tfidf_vect.transform(X_test)

In [19]:
X_train.shape, X_test.shape

((417, 9437), (180, 9437))

In [20]:
# These data sets are "sparse matrix". We can't see them unless we convert using toarray()
X_train

<417x9437 sparse matrix of type '<class 'numpy.float64'>'
	with 29089 stored elements in Compressed Sparse Row format>

In [21]:
# These data sets are "sparse matrix". We can't see them unless we convert using toarray()
X_train.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Latent Semantic Analysis (Singular Value Decomposition)

In [22]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=300, n_iter=10) #n_components is the number of topics, which should be less than the number of features

X_train= svd.fit_transform(X_train)
X_test = svd.transform(X_test)

In [23]:
X_train.shape, X_test.shape

((417, 300), (180, 300))

## Random Forest

In [24]:
from sklearn.ensemble import RandomForestClassifier 

rnd_clf = RandomForestClassifier(n_estimators=100, max_leaf_nodes=16, n_jobs=-1) 
_ = rnd_clf.fit(X_train, y_train)

### Evaluating Model Performance

In [25]:
from sklearn.metrics import accuracy_score

In [26]:
#Train accuracy - Not a good measure of model performance as we are using the same data set to train and test
y_pred_train = rnd_clf.predict(X_train)
acc = accuracy_score(y_train, y_pred_train)
print(f"Train acc: {accuracy_score(y_train, y_pred_train):.4f}")

Train acc: 0.9832


In [27]:
#Test accuracy
y_pred_test = rnd_clf.predict(X_test)
acc = accuracy_score(y_test, y_pred_test)
print(f"Train acc: {accuracy_score(y_test, y_pred_test):.4f}")

Train acc: 0.9056


In [28]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred_test)

array([[47,  0,  5],
       [ 3, 54,  2],
       [ 6,  1, 62]], dtype=int64)

## Stochastic Gradient Descent Classifier

In [29]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(max_iter=100)
_ = sgd_clf.fit(X_train, y_train)

### Evaluating Model Performance

In [30]:
#Train accuracy
y_pred_train = sgd_clf.predict(X_train)
print(f"Train acc: {accuracy_score(y_train, y_pred_train):.4f}")

Train acc: 0.9976


In [31]:
#Test accuracy
y_pred_test = sgd_clf.predict(X_test)
print(f"Train acc: {accuracy_score(y_train, y_pred_train):.4f}")

Train acc: 0.9976


In [32]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred_test)

array([[51,  0,  1],
       [ 2, 57,  0],
       [ 9,  2, 58]], dtype=int64)

## We can see that the accuracy for the both random forest and stochastic gradient descent classifier has been increased:

## Random Forest : 

## Before lemmatisation: 
### Train acc: 0.9760
### Test acc: 0.9111

## After lemmatisation
### Train acc: 0.9880
### Test acc: 0.9167

## Stochastic Gradient Descent Classifier

## Before lemmatisation
### Train acc: 0.9952
### Test acc: 0.9952

## After lemmatisation
### Train acc: 0.9976
### Test acc: 0.9976